In [1]:
import numpy as np
import mrcfile
import platform
PLATFORM_NODE = platform.node()
import sys
if PLATFORM_NODE == 'motel':
    sys.path.insert(0, '/home/sl767/PythonCode/SingleParticleAnalysis')
elif PLATFORM_NODE == 'radon':
    sys.path.insert(0, '/home/zickert/SingleParticleAnalysis')
elif 'lmb' in PLATFORM_NODE:
    sys.path.insert(0, '/lmb/home/schools1/SingleParticleAnalysis')
else:
    raise Exception
from ClassFiles.relion_fixed_it import load_star
from ClassFiles.AdversarialRegularizer import AdversarialRegulariser
from ClassFiles.ut import irfft, rfft
import matplotlib.pyplot as plt
import os
from ClassFiles.ut import cleanStarPath
import itertools
#import subprocess as sp
import argparse
from skimage.measure import compare_ssim as ssim

#%%

In [9]:
if False:
    parser = argparse.ArgumentParser(description='test AR gradient descent')
    parser.add_argument('--gpu', help='GPU to use', required=True)
    parser.add_argument('--positivity', help='AR trained with positivity?', required=True)
    parser.add_argument('--s', help='Sobolev cst', required=True)
    args = vars(parser.parse_args())
    os.environ['CUDA_VISIBLE_DEVICES'] = args['gpu']#'2'
    POS_AR = args['positivity']#'trained_on_non_pos'
    SOBOLEV_CST = args['s']
    PLOT = False
else:
    POS_AR = '0'
    SOBOLEV_CST = '0.0'
    PLOT = False
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#%%

BASE_PATH = '/mnt/datahd/zickert/'
REGULARIZATION_TIK = 1e-3  # For initialization
SAVE_RECO_TO_DISK = False
EVAL_METRIC = 'L2'# 'masked_L2' # 'masked_FSC'
NOISE_LEVEL = '01'
WIN_SIZE = 7 # for SSIM
PRECOND = False
#REGULARIZATION_TIK_GD = 1e6  # During gradient descent

In [3]:
#%%

AR_PATH = '/mnt/datahd/zickert/SPA/Saves/SimDataPaper/Adversarial_Regulariser/no_fancy_data_aug_lr_5e-05_s_{}_pos_{}'.format(SOBOLEV_CST, POS_AR)
regularizer = AdversarialRegulariser(AR_PATH)

POS_AR = ('1' == POS_AR)
    
print('Global parameters:')    
print(SOBOLEV_CST, REGULARIZATION_TIK, PRECOND, REPORT, POS_AR, EVAL_METRIC, NOISE_LEVEL)
print('################################')


def L2(x, GT):    
    return np.sqrt(((x - GT) ** 2).sum())


def vis(data, fourier=True, SCALE=100):
    if fourier:
        data = irfft(data)
    plt.imshow(SCALE*data.squeeze()[..., 45])


def locate_multmap(PDB, noise):
    gt_path = BASE_PATH + 'TrainData/SimDataPaper/Data_0{n}_10k/train/mult_maps/{p}/{p}_mult0{n}.mrc'
    gt_path = gt_path.format(n=noise, p=PDB)
    return gt_path


def locate_star(PDB, noise, iteration):
    star_path =  BASE_PATH + 'TestData/ClassicalRelionData/Data_0{n}_10k/{p}'
    star_path += '/{p}_mult0{n}_it{it}_half1_class001_external_reconstruct.star'
    star_path = star_path.format(p=PDB, n=noise, it=iteration)
    return star_path

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from /mnt/datahd/zickert/SPA/Saves/SimDataPaper/Adversarial_Regulariser/no_fancy_data_aug_lr_5e-05_s_0.0_pos_0/Data/model-10000
Save restored
Global parameters:
0.0 0.001 False 1 False L2 01
################################


In [33]:
REPORT = 1
iterable = itertools.product(['4AIL', '4BTF', '4A2B', '4BB9', '4M82', '4MU9'],
                             ['001', '005'], ['tik'], ['auto'],#5e4, 1e5],
                             [POS_AR], [1], [1e1])
for PDB_ID, IT, INI_POINT, AR_REG_TYPE, POSITIVITY, NUM_GRAD_STEPS, STEP_SIZE_NOMINAL in iterable:   

    star_path = locate_star(PDB_ID, NOISE_LEVEL, IT)
    star_file = load_star(star_path)

    gt_path = locate_multmap(PDB_ID, NOISE_LEVEL) 
    with mrcfile.open(gt_path) as mrc:
        gt = mrc.data.copy()

    ### Is this needed?
    real_data_path = cleanStarPath(star_path, star_file['external_reconstruct_general']['rlnExtReconsDataReal'])
    imag_data_path = cleanStarPath(star_path, star_file['external_reconstruct_general']['rlnExtReconsDataImag'])
    weight_data_path = cleanStarPath(star_path, star_file['external_reconstruct_general']['rlnExtReconsWeight'])
    target_path = cleanStarPath(star_path, star_file['external_reconstruct_general']['rlnExtReconsResult'])
      
    with mrcfile.open(real_data_path) as mrc:
        data_real = mrc.data.copy()
    with mrcfile.open(imag_data_path) as mrc:
        data_im = mrc.data.copy()
    with mrcfile.open(weight_data_path) as mrc:
        kernel = mrc.data.copy()
    with mrcfile.open(target_path) as mrc:
        classical_reco = mrc.data.copy()
       
    complex_data = data_real + 1j * data_im
    
    tikhonov_kernel_init = kernel + REGULARIZATION_TIK
    
       
    if INI_POINT == 'tik':
        init = np.divide(complex_data, tikhonov_kernel_init)
    elif INI_POINT == 'classical':
        init = classical_reco.copy()
        init = rfft(init)
    if POSITIVITY:    
        init = np.fft.rfftn(np.maximum(0, np.fft.irfftn(init)))
    reco = init.copy()

    print('####################')
    print(PDB_ID, NOISE_LEVEL, IT, INI_POINT, AR_REG_TYPE, POSITIVITY, NUM_GRAD_STEPS, STEP_SIZE_NOMINAL)
#    print('####################')

    if EVAL_METRIC == 'L2':
        pass
#        print('INIT L2: ', L2(irfft(init), gt))
    elif EVAL_METRIC == 'L2_and_SSIM':
        print('INIT L2: ', L2(irfft(init), gt), '. INIT SSIM: ', ssim(irfft(init), gt, win_size=WIN_SIZE))       
    
    STEP_SIZE = STEP_SIZE_NOMINAL
    for k in range(NUM_GRAD_STEPS):
        gradient = regularizer.evaluate(reco)
        reco_o = reco
        reco = reco - STEP_SIZE * gradient  
        if POSITIVITY:    
            reco = np.fft.rfftn(np.maximum(0, np.fft.irfftn(reco)))
        
        if k % REPORT == 0:
            if EVAL_METRIC == 'L2':
                print('Relative L2 Diff: ', (L2(irfft(reco), gt) - L2(irfft(init), gt))/L2(np.zeros_like(gt), gt))
            elif EVAL_METRIC == 'L2_and_SSIM':
                print('L2 ', L2(irfft(reco), gt), '. SSIM: ', ssim(irfft(reco), gt, win_size=WIN_SIZE))  
        if PLOT:
            plt.figure(k+1, figsize=(20, 3))
            plt.subplot(141)
            vis(init)
            plt.colorbar()
            plt.subplot(142)
            vis(reco)
            plt.colorbar()
            plt.subplot(143)
            vis(gradient)
            plt.colorbar()
            plt.subplot(144)
            vis(reco-init)
            plt.colorbar()
            plt.show()


####################
4AIL 01 001 tik auto 0 1 10.0
Relative L2 Diff:  6.12588e-06
####################
4AIL 01 005 tik auto 0 1 10.0
Relative L2 Diff:  1.40225e-05
####################
4BTF 01 001 tik auto 0 1 10.0
Relative L2 Diff:  1.6049e-06
####################
4BTF 01 005 tik auto 0 1 10.0
Relative L2 Diff:  4.01224e-07
####################
4A2B 01 001 tik auto 0 1 10.0
Relative L2 Diff:  -1.95365e-06
####################
4A2B 01 005 tik auto 0 1 10.0
Relative L2 Diff:  -1.81411e-06
####################
4BB9 01 001 tik auto 0 1 10.0
Relative L2 Diff:  1.152e-05
####################
4BB9 01 005 tik auto 0 1 10.0
Relative L2 Diff:  1.5826e-05
####################
4M82 01 001 tik auto 0 1 10.0
Relative L2 Diff:  7.30211e-07
####################
4M82 01 005 tik auto 0 1 10.0
Relative L2 Diff:  7.30211e-06
####################
4MU9 01 001 tik auto 0 1 10.0
Relative L2 Diff:  1.32625e-06
####################
4MU9 01 005 tik auto 0 1 10.0
Relative L2 Diff:  7.39907e-06


In [5]:
print(L2(np.zeros_like(gt),gt))

3.41561
